In [1]:
import os
import openai

In [2]:
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import AsyncOpenAI

In [4]:
from llama_index import SimpleDirectoryReader

docs = SimpleDirectoryReader(
    input_files = ["./ce-little-red-story-pcp-115014.pdf"]
).load_data() # one doc per page 

# Basic RAG Pipeline

In [5]:
from llama_index import Document

doc = Document(text = '\n\n'.join([doc.text for doc in docs])) #merge the docs into one doc

In [6]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model = 'gpt-3.5-turbo', temperature = 0.1)
service_context = ServiceContext.from_defaults(
            llm = llm,
            embed_model = 'local:BAAI/bge-small-en-v1.5' # hugging face model
)
index = VectorStoreIndex.from_documents([doc], service_context = service_context)

In [7]:
query_engine = index.as_query_engine() # send user query and compare it against data

In [8]:
response = query_engine.query(
    'What is the story about?')

In [9]:
print(str(response))

The story is about a little girl named Little Red Riding Hood who is sent by her mother to take a basket of sweet cherry pies to her grandmother's house. Along the way, she encounters a thorn that traps her in her scarlet cloak. A mysterious figure helps her free from the thorn, but Little Red Riding Hood doesn't realize that it is the Big Bad Wolf. She continues on her journey and arrives at her grandmother's cottage, only to find that the wolf has already eaten her grandmother and disguised himself as her. Little Red Riding Hood realizes the truth and screams for help. The Wise Owl sends the Storyland Vets, who put the wolf to sleep and rescue the grandmother. In the end, the wolf is left with the grandmother's two pointy shoes inside him, which cause him discomfort whenever he feels hungry.


# Evaluating setup using TruLens

## a set of questions to evaluate the model against

In [10]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        item = line.strip() #remove new line character and convert it to integer
        print(item)
        eval_questions.append(item)

What is the name of the main character?
What was in Red Riding Hood basket?
Whom did Red Riding Hood went to visit?
Where did the grandma live?
Who ate the grandma?
Who helped Red Riding Hood to rescue the grandma?
What happened to the wolf at the end?


In [11]:
new_question = 'what was the morale of the story?'
eval_questions.append(new_question)
eval_questions

['What is the name of the main character?',
 'What was in Red Riding Hood basket?',
 'Whom did Red Riding Hood went to visit?',
 'Where did the grandma live?',
 'Who ate the grandma?',
 'Who helped Red Riding Hood to rescue the grandma?',
 'What happened to the wolf at the end?',
 'what was the morale of the story?']

## Setting up the model

In [12]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [13]:
from utils_trulens import get_prebuilt_trulens_recorder
tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id = 'Direct Query Engine')

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [14]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [15]:
records, feedback = tru.get_records_and_feedback(app_ids = [])

In [16]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2b3dee07675cc3ef7d46ae34a5c7dccf,"""What is the name of the main character?""","""The name of the main character is Little Red ...",-,"{""record_id"": ""record_hash_2b3dee07675cc3ef7d4...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-21T11:49:30.209652"", ""...",2023-12-21T11:49:34.257887,1.0,0.0,1.0,[{'args': {'prompt': 'What is the name of the ...,[{'args': {'prompt': 'What is the name of the ...,[{'args': {'source': 'www.scholastic.co.ukhttp...,4,1124,0.001692
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9f0361836e93d14d61b93c29a8f6afdf,"""What was in Red Riding Hood basket?""","""A basket of sweet cherry pies was in Red Ridi...",-,"{""record_id"": ""record_hash_9f0361836e93d14d61b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-21T11:49:34.810574"", ""...",2023-12-21T11:49:38.955101,1.0,0.0,1.0,[{'args': {'prompt': 'What was in Red Riding H...,[{'args': {'prompt': 'What was in Red Riding H...,[{'args': {'source': 'www.scholastic.co.ukhttp...,4,1125,0.001695
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7bff025cb72caa23e964784eef13b580,"""Whom did Red Riding Hood went to visit?""","""Red Riding Hood went to visit her Grandma.""",-,"{""record_id"": ""record_hash_7bff025cb72caa23e96...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-21T11:49:39.378076"", ""...",2023-12-21T11:49:43.681415,1.0,0.0,1.0,[{'args': {'prompt': 'Whom did Red Riding Hood...,[{'args': {'prompt': 'Whom did Red Riding Hood...,[{'args': {'source': 'www.scholastic.co.ukhttp...,4,1122,0.001687
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_43da5281542f73ae11f15260bb497b05,"""Where did the grandma live?""","""The grandma lived at the other end of Long-Lo...",-,"{""record_id"": ""record_hash_43da5281542f73ae11f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-21T11:49:44.584971"", ""...",2023-12-21T11:49:48.941005,1.0,0.0,1.0,[{'args': {'prompt': 'Where did the grandma li...,[{'args': {'prompt': 'Where did the grandma li...,[{'args': {'source': 'www.scholastic.co.ukhttp...,4,1130,0.001705
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1df5cb17c6496cfde3fb4c7cd6bad868,"""Who ate the grandma?""","""The Big Bad Wolf ate the grandma.""",-,"{""record_id"": ""record_hash_1df5cb17c6496cfde3f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-21T11:49:49.385510"", ""...",2023-12-21T11:49:53.372515,1.0,0.0,1.0,"[{'args': {'prompt': 'Who ate the grandma?', '...","[{'args': {'prompt': 'Who ate the grandma?', '...",[{'args': {'source': 'www.scholastic.co.ukhttp...,3,1116,0.001678


In [17]:
tru.get_leaderboard(app_ids = [])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
Direct Query Engine,0.0,1.0,0.857143,4.0,0.001707


In [18]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://localhost:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

# Advanced RAG pipeline

## Sentence window retrieval

In [19]:
from llama_index.llms import OpenAI

llm = OpenAI(model = 'gpt-3.5-turbo', temperature = 0.1)

In [20]:
from utils_trulens import build_sentence_window_index
# retreive multiple sentences instead of one
sentence_index = build_sentence_window_index(
    doc,
    llm,
    embed_model = 'local:BAAI/bge-small-en-v1.5',
    save_dir = 'sentence_index'
)

In [21]:
from utils_trulens import get_sentence_window_query_engine
sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [22]:
response = sentence_window_engine.query(
    'what was the morale of the story?'
)
print(str(response))

The morale of the story is to be cautious and not trust strangers, as they may have ulterior motives.


In [23]:
response = query_engine.query(
    'what was the morale of the story?'
)
print(str(response))

The morale of the story is to never stop and speak to the Big Bad Wolf.


In [24]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = 'Sentence window query engine'
)

In [25]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What is the name of the main character?
Little Red Riding Hood
What was in Red Riding Hood basket?
Sweet cherry pies.
Whom did Red Riding Hood went to visit?
Red Riding Hood went to visit her Grandma.
Where did the grandma live?
The grandma lived in a cottage.
Who ate the grandma?
The Big Bad Wolf ate the grandma.
Who helped Red Riding Hood to rescue the grandma?
There is no information provided in the given context about who helped Red Riding Hood rescue her grandma.
What happened to the wolf at the end?
The wolf was sewn up again after the Storyland Vets found Grandma safe and well inside his tummy. However, they accidentally left her two pointy shoes inside him.
what was the morale of the story?
The moral of the story is to be cautious and not talk to strangers.


In [26]:
tru.get_leaderboard(app_ids = [])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
Sentence window query engine,0.071429,0.833333,0.885714,4.875,0.000603


## Auto_merging retrieval

In [28]:
from utils_trulens import build_automerging_index

automerging_index = build_automerging_index(
        docs,
        llm,
        embed_model="local:BAAI/bge-small-en-v1.5",
        save_dir="merging_index"
        )

In [29]:
from utils_trulens import get_automerging_query_engine
automerging_query_engine = get_automerging_query_engine(
    automerging_index
)

In [30]:
response = automerging_query_engine .query(
    'what was the morale of the story?'
)
print(str(response))

> Merging 4 nodes into parent node.
> Parent node id: 5e1e21cd-474e-425b-8054-46fc312af0ca.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: d8868ccb-da68-470e-b752-b6544fdbe3fe.
> Parent node text: “Let me in, Grandma,” she called merrily. “I have brought you a basket of sweet cherry pies.”
 “L...

> Merging 5 nodes into parent node.
> Parent node id: 374b0bb9-3339-43c0-904e-5eff4ace2d9a.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
Fairytale Kingdom
The Story of Little Red Riding...

> Merging 1 nodes into parent node.
> Parent node id: daeae9c7-0679-4091-aabd-c4296e87d252.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: 7cf2567c-9525-419a-8072-d6efb6cf9895.
> Parent node text: www.scholastic.co.ukhttp://www.s

In [31]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [32]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x21f2f52d750 is calling an instrumented method <function BaseQueryEngine.query at 0x0000021F100F7C70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x21f28e3ea10) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x21f2f52d750 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000021F138E5900>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x21f28e3ea10) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x21f2f52feb0 is calling an instrumented method <function BaseRetriever.retrieve at 0x0000021F100F6EF0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obje

> Merging 5 nodes into parent node.
> Parent node id: 374b0bb9-3339-43c0-904e-5eff4ace2d9a.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
Fairytale Kingdom
The Story of Little Red Riding...

> Merging 4 nodes into parent node.
> Parent node id: 5e1e21cd-474e-425b-8054-46fc312af0ca.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: d8868ccb-da68-470e-b752-b6544fdbe3fe.
> Parent node text: “Let me in, Grandma,” she called merrily. “I have brought you a basket of sweet cherry pies.”
 “L...

> Merging 2 nodes into parent node.
> Parent node id: 7cf2567c-9525-419a-8072-d6efb6cf9895.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
Fairytale Kingdom
The Story of Little Red Riding...

> Merging 1 nodes into parent node.
> Parent node id: daeae9c7-0679-4091-aabd-c4296e87d252.
> Parent node text: www.scholastic.co.ukhttp://www.s

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x21f2f52d780 is calling an instrumented method <function BaseSynthesizer.synthesize at 0x0000021F10DE5B40>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x21f23665ff0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x21f2f52d780 is calling an instrumented method <function CompactAndRefine.get_response at 0x0000021F10DE5000>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x21f23665ff0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x21f2f52d780 is calling an instrumented method <function Refine.get_response at 0x0000021F1173DC60>. The path of this call may be incorrect.
Guessing path

What is the name of the main character?
The main character's name is Little Red Riding Hood.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x21f2f52d750 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000021F138E5900>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x21f28e3ea10) using this function.


...

What happened to the wolf at the end?
The wolf was put to sleep by the Storyland Vets and Grandma was found safe and well inside his tummy. However, when the wolf was sewn up again, Grandma's two pointy shoes were accidentally left inside him. Now, whenever the wolf feels hungry, the shoes dance and prance, causing him to howl, and he never dreams of eating a grandma again.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x21f2f52d750 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000021F138E5900>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x21f28e3ea10) using this function.


> Merging 4 nodes into parent node.
> Parent node id: 5e1e21cd-474e-425b-8054-46fc312af0ca.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: d8868ccb-da68-470e-b752-b6544fdbe3fe.
> Parent node text: “Let me in, Grandma,” she called merrily. “I have brought you a basket of sweet cherry pies.”
 “L...

> Merging 5 nodes into parent node.
> Parent node id: 374b0bb9-3339-43c0-904e-5eff4ace2d9a.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
Fairytale Kingdom
The Story of Little Red Riding...

> Merging 1 nodes into parent node.
> Parent node id: daeae9c7-0679-4091-aabd-c4296e87d252.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: 7cf2567c-9525-419a-8072-d6efb6cf9895.
> Parent node text: www.scholastic.co.ukhttp://www.s

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x21f2f52d780 is calling an instrumented method <function CompactAndRefine.get_response at 0x0000021F10DE5000>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x21f23665ff0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x21f2f52d780 is calling an instrumented method <function Refine.get_response at 0x0000021F1173DC60>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x21f23665ff0) using this function.


what was the morale of the story?
The morale of the story is to never stop and speak to the Big Bad Wolf.


In [33]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.0,1.0,1.0,5.625,0.001772
